<a href="https://colab.research.google.com/github/freguti/ProgettoDataSpaces/blob/main/Progetto_DataSpaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Analysis of "Predicting Pulsar Star"**

Simone Soncin - s263094

DA FINIRE

  Strumenti Utilizzati

  Rivedere lettura grafici

  VEDERE SE È IL CASO DI LASCIARE IL RATIO ORIGINALE.

  Classificazione

  Conclusioni


##**Indice**

 

1. [Introduzione](#Introduzione)
2. [Analisi Dataset](#Analisi)
3. [PCA](#PCA)
4. [Bilanciamento Datadet](#bilanciamento)
5. [Classificazione](#classificazione)
  *   [Logistic Regression](#lr)
  *   [Support Vector Machine](#svm)
  *   [Naïve Bayes Classifier](#nbc)
  *   [Random Forest Classifier](#rfc)
  *   [K-Neares Neighbors Classifier](#knn)
6. [Conclusioni](#conclusioni)










In [ ]:
#Show/Hide code button
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">Il codice è stato nascosto per permettere una lettura più fluida. Per mostrarlo cliccare qui<br><input type="submit" value="Abilita/Disabilita codice"></form>''')

<a name="introduzione"></a>
##**Introduzione**
In questa tesina mi dedicherò ad analizzare un dataset, manipolare i dati per utilizzarli in maniera corretta e successivamente applicare dei modelli predittivi, identificando quale si comporta meglio nel nostro caso di studio.

Le stelle pulsar sono delle stelle di neutroni che emettono un potente segnale radio dai poli misurabile dalla terra. Una stella pulsar ruota ad altissima velocità, rendendo questo segnale radio misurabile a tratti e fornendogli un pattern periodico che varia per ogni stella pulsar.

Il dataset che utilizzerò è disponibile al link: https://www.kaggle.com/colearninglounge/predicting-pulsar-starintermediate?select=pulsar_data_train.csv 

Questo dataset contiene una serie di parametri continui discretizzati, i cui primi 4 sono derivati dal "integrated pulse profile", mentre gli ultimi 4 parametri sono ottenuti in maniera analoga dalla "DM-SNR curve".

- Integrated profile: è ottenuto dal segnale periodico emesso dalla pulsar ed è univoco per ognuna di esse. Per questo motivo è utilizzato per identificarle a discapito del loro nome. 
- DM-SNR Curve: Dispersion Measure of the Signal to Noise Ratio in poche parole è la densità degli elettroni liberi lungo la linea ottica.

I nostri parametri sono stati ottenuti da questi due dati, applicando rispettivamente la media, la deviazione standard, la curtosi (indica quanto sono spesse le code della distribuzione) e la simmetria statistica (asimmetria della distribuzione di probabilità rispetto alla sua media).

Ecco la lista dettagliata dei parametri:
- **Mean of the integrated profile**
- **Standard deviation of the integrated profile**
- **Excess kurtosis of the integrated profile**
- **Skewness of the integrated profile**
- **Mean of the DM-SNR curve**
- **Standard deviation of the DM-SNR curve**
- **Excess kurtosis of the DM-SNR curve**
- **Skewness of the DM-SNR curve**

Infine è presente un ultimo parametro discreto che indica se il segnale appartiene ad una pulsar (target_class = 1) o no (target_class = 0).

###**Strumenti Utilizzati**

- Grafici: Per i grafici è stata utilizzata la libreria chart studio, che permette di creare dei grafici interattivi in maniera semplice ed intuitiva. Questo strumento ha il limite che non consente un upload di dati superiore ai 500Kb. Per la creazione dei grafici a istogramma e a boxplot, ho dovuto dividere il grafico desiderato in 4 grafici più piccoli a causa del raggiungimento di tale limite. Ho provveduto ad abbinare le feature in base al calcolo statistico usato per l'estrazione dalla grandezza fisica misurata.

##Utility

In [4]:
#Capture removes the cell's output
%%capture
#Imports
import pandas as pd
import seaborn
import numpy as np
import functools
import ipywidgets as widgets
import sys
from google.colab import drive
!pip install -U imbalanced-learn
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
!pip install nbconvert
!pip install -q gwpy
import plotly
!pip install chart_studio
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn.metrics import roc_auc_score
from numpy import mean
from scipy.cluster import hierarchy as hc
from sklearn.decomposition import PCA

#alla fine sarà da eseguire alla fine del notebook
#%%capture
#It's used to convert this notebook in HTML
#!jupyter nbconvert --to html /content/drive/MyDrive/DataSpaces/Progetto_DataSpaces.ipynb

In [5]:
#Parameters
%%capture
chart_studio.tools.set_credentials_file(username='freguti', api_key='4T2iyEX3eZP4aJ8PEJye')
HISTOGRAM_COLORS = {"pulsar" : "#388004",
                    "not_pulsar" : "#8B0000"}
drive.mount('/content/drive')
BOX_OPACITY = 0.3
COLOR_PALETTE = seaborn.color_palette("Blues_d").as_hex()
COLORSCALE_HEATMAP = [         [0.0, '#011f4b'],
                [0.1111111111111111, '#03396c'], 
                [0.2222222222222222, '#005b96'], 
                [0.3333333333333333, '#2171b5'], 
                [0.4444444444444444, '#6497b1'], 
                [0.5555555555555556, '#6baed6'], 
                [0.6666666666666666, '#B0E2FF'], 
                [0.7777777777777778, '#b3cde0'], 
                [0.8888888888888888, '#bdd7e7'], 
                               [1.0, '#BFEFFF']] 
PALETTE_HEATMAP = [[0.0, '#F5FFFA'], 
                         [0.2, '#ADD8E6'], 
                         [0.4, '#87CEEB'],
                         [0.6, '#87CEFA'], 
                         [0.8, '#40E0D0'], 
                         [1.0, '#00CED1']]

LR_PARAM = [{
    'clf__solver': ['liblinear'],
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'clf__penalty': ['l2', 'l1']
},
{
    'clf__solver': ['newton-cg', 'lbfgs'], 
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2']
}]
SVM_PARAM = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10, 100],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': [0.1, 1, 10, 100],
}]

RFC_PARAM = {
    'clf__max_depth': [50, 75, 100],
    'clf__max_features': ["sqrt", "log2"],
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500]
}

KNN_PARAM = {
    'clf__n_neighbors': [2, 3, 5, 10, 15],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}

NBC_PARAM = {}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWj3jX47d_k4vX9QZVO_OAZSjWe029I51p789XQ3FVSWYOiIqZv7i30


In [17]:
%%capture
#Graph utility

def histogram_bar(tipo, data, col, visible=True, norm = ""):
    
    color = HISTOGRAM_COLORS[tipo]
    
    data_16 = data.astype(np.float16 ,copy=True)
    
    return go.Histogram(
        x = data_16[col],
        name = tipo,
        marker = dict(color = color),
        visible = visible,
        opacity = 1,
        histnorm=norm
    )

#fare l'istogramma con chart studio https://plotly.com/python/getting-started-with-chart-studio/
def feature_histogram(dataset, title = "", feature = ""):

  trace0 = histogram_bar("not_pulsar", dataset[dataset['target_class'] == 0], feature)
  trace1 = histogram_bar("pulsar", dataset[dataset['target_class'] == 1], feature)
  
  data = [trace0, trace1]
  
  layout = dict(
      title=title,
      autosize=True,
      yaxis=dict(
          title='value',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
      barmode='group',
      bargap=0.15,
      bargroupgap=0.1
  )
  fig = dict(data=data, layout=layout)
  return py.iplot(fig, filename=title)


def on_slider_change(sender,dataset):
  if sender['name'] == 'value':
    selected_option = sender['owner'].value
    print(selected_option)
    feature_histogram(dataset = dataset, title = "Distribuzione features", feature = selected_option)

def create_slider_controlled_histogram(col,pulsar,not_pulsar,first_plot):
  pulsar_plot = []
  not_pulsar_plot = []
  hist_features,pulsar_plot,not_pulsar_plot = prepare_set(dataset.columns,first_plot,pulsar,not_pulsar)
  active_index = 0

  hist_not = [(histogram_bar('not_pulsar', not_pulsar_plot, col, False,'percent') if i != active_index
            else histogram_bar('not_pulsar', not_pulsar_plot, col, True,'percent'))
              for i, col in enumerate(hist_features)
              ]
  hist_pulsar = [(histogram_bar('pulsar', pulsar_plot, col, False,'percent') if i != active_index
            else histogram_bar('pulsar', pulsar_plot, col, True,'percent'))
              for i, col in enumerate(hist_features)
              ]

  total_data = hist_not + hist_pulsar
  number_of_features = len(hist_features)
  steps = []

  for i in range(number_of_features):
      step = dict(
          method = 'restyle',
          args = ['visible', [False] * number_of_features],
          label = hist_features[i],
      )
      step['args'][1][i] = True
      steps.append(step)
      
  sliders = [dict(
      active = active_index,
      currentvalue = dict(
          prefix = "Feature: ", 
          xanchor= 'center',
      ),
      pad = {"t": 50},
      steps = steps,
      len=1,
  )]

  layout = dict(
      sliders=sliders,
      autosize=True,
      yaxis=dict(
          title='valori',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
  )

  fig = dict(data=total_data, layout=layout)

  return py.iplot(fig, filename='bar_slider')  

def create_box(type, data, col, visible=False):
   
    c = HISTOGRAM_COLORS[type]
    data_16 = data.astype(np.float16 ,copy=True)
    return go.Box(
        y = data_16[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = BOX_OPACITY,
    )

def create_slider_controlled_Boxplot(col,pulsar,not_pulsar,first_plot):
  pulsar_plot = []
  not_pulsar_plot = []
  box_features,pulsar_plot,not_pulsar_plot = prepare_set(dataset.columns,first_plot,pulsar,not_pulsar)
  active_index = 0

  box_not_pulsar = [(create_box('not_pulsar', not_pulsar_plot, col, False) if i != active_index
            else create_box('not_pulsar', not_pulsar_plot, col, True))
              for i, col in enumerate(box_features)
              ]
  box_pulsar = [(create_box('pulsar', pulsar_plot, col, False) if i != active_index
              else create_box('pulsar', pulsar_plot, col, True))
              for i, col in enumerate(box_features)
              ]

  data = box_not_pulsar + box_pulsar
  number_of_features = len(box_features)
  steps = []

  for i in range(number_of_features):
      step = dict(
          method = 'restyle',  
          args = ['visible', [False] * number_of_features],
          label = box_features[i],
      )
      step['args'][1][i] = True # Toggle i'th trace to "visible"
      steps.append(step)
      
  sliders = [dict(
      active = active_index,
      currentvalue = dict(
          prefix = "Feature: ", 
          xanchor= 'center',
      ),
      pad = {"t": 50},
      steps = steps,
      len=1,
  )]

  layout = dict(
      sliders=sliders,
      autosize=True,
      yaxis=dict(
          title='valori',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
  )

  fig = dict(data=data, layout=layout)
  return py.iplot(fig, filename='box_slider')

def prepare_set(col,first_plot,pulsar,not_pulsar): 
  pulsar_plot = pulsar
  not_pulsar_plot = not_pulsar
  box_features = col.values.tolist()
  if first_plot == "mean":
    #to_drop = [box_features[1],box_features[2],box_features[3],box_features[5],box_features[6],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[0],box_features[4]]
  if first_plot == "deviation":
    #to_drop = [box_features[0],box_features[2],box_features[3],box_features[4],box_features[6],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[1],box_features[5]]
  if first_plot == "kurtosis":
    #to_drop = [box_features[0],box_features[1],box_features[3],box_features[4],box_features[5],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[2],box_features[6]]
  if first_plot == "skewness":
    #to_drop = [box_features[0],box_features[1],box_features[2],box_features[4],box_features[5],box_features[6],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[3],box_features[7]]
  return hist_features,pulsar_plot,not_pulsar_plot

def plot_variance(pca, title):
  tot_var = np.sum(pca.explained_variance_)
  ex_var = [(i / tot_var) * 100 for i in sorted(pca.explained_variance_, reverse=True)]
  cum_ex_var = np.cumsum(ex_var)

  cum_var_bar = go.Bar(
      x=list(range(1, len(cum_ex_var) + 1)), 
      y=ex_var,
      name="Varianza di ogni componente",
      marker=dict(
          color=HISTOGRAM_COLORS["pulsar"],
      ),
      opacity= 1
      )

  variance_line = go.Scatter(
      x=list(range(1, len(cum_ex_var) + 1)),
      y=cum_ex_var,
      mode='lines+markers',
      name="Varianza cumulativa",
      marker=dict(
          color=HISTOGRAM_COLORS["not_pulsar"],
      ),
      opacity= 1,
      line=dict(
          shape='hv',
      ))
  data = [cum_var_bar, variance_line]
  layout = go.Layout(
      autosize=True,
      title=title,
      yaxis=dict(
          title='Varianza (%)',
      ),
      xaxis=dict(
          title="Componenti principali",
          dtick=1,
          rangemode='nonnegative'
      ),
      legend=dict(
          x=0,
          y=1,
      ),
  )
  fig = go.Figure(data=data, layout=layout)
  return py.iplot(fig, filename=title)

def PCA_Reduction(X_norm, plot = False , n_components = 8):
  pca = PCA(random_state=42) #random seed
  ortogonal = pca.fit_transform(X_norm)

  if plot:
      p = plot_variance(pca, "Varianza singola e cumulativa")

  pca.components_ = pca.components_[:n_components]
  reduced_data = np.dot(ortogonal, pca.components_.T)
  X_red = pd.DataFrame(reduced_data, columns=["PC#%d" % (x + 1) for x in range(n_components)])
  if plot:
      return p, X_red
  else:
      return X_red

def grid_search_cv(model, params, X_train, y_train, cv):
  over = SMOTE(sampling_strategy=0.4)
  under = RandomUnderSampler(sampling_strategy=0.6)
  steps = [('over', over), ('under', under), ('model', model)]
  pipeline = Pipeline(steps=steps)
  grid_search = GridSearchCV(estimator=pipeline, 
                              param_grid=params, 
                              cv=cv, 
                              n_jobs=-1,       # Use all processors
                              scoring='f1',    # Use f1 metric for evaluation
                              return_train_score=True)
  grid_search.fit(X_train, y_train)
  return grid_search

<a name="Analisi"></a>
##**Analisi Dataset**

Come prima cosa è necessario eseguire un'analisi del dataset, eseguendo eventualmente delle operazioni per migliorarlo e renderlo più adatto al nostro tipo di analisi.

La fase di analisi è la più importante, perchè permette di pulire il dataset migliorando l'analisi effettuata e ridurre la potenza di calcolo necessaria per eseguirla.



In [11]:
dataset = pd.read_csv('/content/drive/My Drive/DataSpaces/pulsar_stars.csv')
for column in dataset:
 if dataset[column].dtype == 'float64':
  dataset[column]=pd.to_numeric(dataset[column], downcast='float')
  if dataset[column].min() <= np.finfo(np.float16).min or dataset[column].max() >= np.finfo(np.float16).max:
    raise CustomError("Range Exception")
 if dataset[column].dtype == 'int64':
  dataset[column]=pd.to_numeric(dataset[column], downcast='integer')

print("il dataset ha %d record e %d features\n\nNumero di valori null per ogni feature:" % dataset.shape)
print(dataset.isnull().sum())

print("\nDa quel che si può vedere, non ci sono valori nulli all'interno del nostro dataset. Nel caso in cui ne avessimo trovato uno avremo avuto due possibilità:\n")
print("- Se i valori nulli non sono molti possiamo procedere eliminando i record che riportano il valore nullo;\n- Se il numero di questi valori nulli è elevato si può procedere alla sostituzione con dei valori di default\n")

print("ci sono 2 classi differenti.\n0 -> non è una pulsar\n1 -> è una pulsar")
X, y = make_classification(n_classes = 2, )
target_class = dataset['target_class'].value_counts()
print("numero di record per ogni classe:\n" + target_class.to_string() + "\n")

il dataset ha 17898 record e 9 features

Numero di valori null per ogni feature:
 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64

Da quel che si può vedere, non ci sono valori nulli all'interno del nostro dataset. Nel caso in cui ne avessimo trovato uno avremo avuto due possibilità:

- Se i valori nulli non sono molti possiamo procedere eliminando i record che riportano il valore nullo;
- Se il numero di questi valori nulli è elevato si può procedere alla sostituzione con dei valori di default

ci sono 2 classi differenti.
0 -> non è una pulsar
1 -> è una pulsar
numero 

La prima analisi che si dovrà effettuare è quella che permette di verificare i tipi di dati del dataset

In [ ]:
dataset.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683781,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242226,0
1,102.507812,58.882431,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393578,0
2,103.015625,39.341648,0.323328,1.051164,3.121238,21.744669,7.735822,63.171909,0
3,136.750000,57.178448,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593662,0
4,88.726562,40.672226,0.600866,1.123492,1.178930,11.468719,14.269573,252.567307,0


Possiamo notare che tutte le features tranne la classe sono reali, perchè sono il prodotto di calcoli statistici su dei fenomeni fisici e quindi intrinsecamente continui.
La classe di appartenenza invece è discreta, e visto che può assumere due valori possiamo dichiarare che il nostro problema consiste in una classificazione di tipo binario.

Un'altra analisi preliminare che possiamo eseguire consiste nell'analizzare dei dati statistici relativi ai nostri campi del dataset. Questa analisi estrapolerà questi valori:

- **count:** specifica il numero dei record presenti nel dataset
- **mean:** specifica la media dell'attributo calcolata per tutti i record
- **std:** specifica la deviazione standard dell'attributo
- **min:** specifica il valore minimo dell'attributo
- **25%:** il 25% dei record ha un valore minore di quello visualizzato (lower percentile)
- **50%:** il 50% dei record ha un valore minore di quello visualizzato (median percentile)
- **75%:** il 75% dei record ha un valore minore di quello visualizzato (upper percentile)
- **max:** specifica il valore massimo dell'attributo

In [ ]:
dataset.describe()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079773,46.549370,0.477856,1.770286,12.614424,26.326546,8.303542,104.857712,0.091574
std,25.652920,6.843179,1.064041,6.167913,29.472887,19.470539,4.506091,106.514359,0.288432
min,5.812500,24.772041,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376019,0.027098,-0.188572,1.923077,14.437331,5.781506,34.960505,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461315,8.433515,83.064556,0.000000
75%,127.085938,51.023203,0.473325,0.927783,5.464256,28.428104,10.702959,139.309326,0.000000
max,192.617188,98.778908,8.069522,68.101624,223.392136,110.642212,34.539845,1191.000854,1.000000


###Grafici

####Istogramma

In [8]:
feature_histogram(dataset = dataset, title = "Distribuzione delle classi", feature = "target_class")

Da quel che possiamo vedere il dataset che abbiamo è molto sbilanciato. Per ogni record di classe "1" esistono 10 record di classe "0".

Questo sbilanciamento potrebbe portare il modello predittivo ad avere dei bias, portandolo a predirre più frequentemente la classe con più campioni solamente perchè è stata più presente in fase di train.

Gli Istogrammi sono molto utili per vedere la distribuzione delle classi per ogni feature in relazione al suo valore.
Tramite uno slider è possibile visualizzare delle features diverse.

In [12]:
pulsar = dataset[dataset['target_class'] == 1]
not_pulsar = dataset[dataset['target_class'] == 0]

create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar, "mean")

[' Mean of the integrated profile', ' Mean of the DM-SNR curve']


In [13]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"deviation")

[' Standard deviation of the integrated profile', ' Standard deviation of the DM-SNR curve']


In [14]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"kurtosis")

[' Excess kurtosis of the integrated profile', ' Excess kurtosis of the DM-SNR curve']


In [15]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"skewness")

[' Skewness of the integrated profile', ' Skewness of the DM-SNR curve']


A causa di un limite dello strumento, che consente un upload limitato di dati, ho dovuto dividere il grafico in 4. Ho optato per separare i grafici per metodo statistico applicato alle due misurazioni.

ho usato una normalizzazione percentuale che mi mostra, per ogni classe, la distribuzione percentuale dei valori. L'ho fatto a causa dell'elevata differenza numerica tra le due classi, che non mi permetteva di mostrare correttamente i valori assoluti.

Si può notare che la distribuzione dei valori "non pulsar" tende ad essere la maggior parte delle volte una distribuzione Gaussiana, e ogni tanto una distribuzione di Poisson.

La distribuzione dei valori "pulsar" tende ad essere più costante e disordinata, evitando una concentrazione di valori. Questo deriva dalle nostre supposizioni iniziali, che indicavano come univoci i pattern di ogni pulsar.

Questa differenza può essere significativa in fase di classificazione, perchè le varie features tendono a sovrapporsi poche volte, ciò implica che potenzialmente tutte le classi possono essere utili per determinare la classe.

####Boxplot

In [18]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "mean")

In [19]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "deviation")

In [20]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "kurtosis")

In [21]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "skewness")

Sembra che tutti i parametri siano rilevanti nella classificazione. Nessuno di loro ha una distribuzione media troppo simile alla controparte della classe opposta. 

####Matrice di correlazione

Una matrice di correlazione è una tabella che mostra i coefficienti di correlazione tra insiemi di variabili. Ogni variabile casuale ( $X_i$ ) nella tabella è correlata con ciascuno degli altri valori nella tabella ( $X_j$ ); questo permette di vedere quali coppie hanno la più alta correlazione. La correlazione si riferisce a qualsiasi associazione statistica, ma nell'uso comune del termine si indica quanto due variabili siano vicine ad avere una relazione lineare l'una con l'altra.

Nel grafico sottostante abbiamo utilizzato la correlazione di Pearson, che misura la correlazione lineare tra due variabili statistiche X e Y, ritornando un valore compreso tra -1 e 1.
- Se il valore è vicino a -1 vuol dire che le due variabili hanno una forte correlazione negativa, cioè al crescere della prima decresce la seconda;
- Se il valore è vicino a 1 vuol dire che le due variabili hanno una forte correlazione, cioè al crescere della prima crescerà anche la seconda;
- Se il valore è vicino allo 0 le due variabili sono scorrelate

Questo valore viene calcolato così: $Corr_{i,j} = \frac{cov(X,Y)}{\sigma_x\sigma_y}$

dove cov è la covarianza tra i due valori e $\sigma$ è lo scarto quadratico medio.

In [ ]:
import pandas as pd

corr = dataset.corr(method='pearson')
#corr.style.background_gradient(cmap='coolwarm').set_precision(2)
title = "Correlation Matrix"

z_text = np.around(corr.values.tolist(), decimals=2)

figure = ff.create_annotated_heatmap(z=corr.values, 
                                         x=corr.columns.tolist(), 
                                         y=corr.index.tolist(),
                                         annotation_text=z_text,
                                         colorscale=PALETTE_HEATMAP,
                                         showscale=True)

figure.layout.title = title
figure.layout.autosize = False
figure.layout.width = 850
figure.layout.height = 850
figure.layout.margin = go.layout.Margin(l=140, r=100, b=200, t=80)
figure.layout.xaxis.update(side='bottom')
figure.layout.yaxis.update(side='left')

for i in range(len(figure.layout.annotations)):
    figure.layout.annotations[i].font.size = 8
                                
py.iplot(figure, filename=title)

Possiamo notare che sia la skewness e la kurtosis (0,95 per l'integrated profile e 0,92 per la DM-SNR curve) che la mean e la standard deviation (0,55 per l'integrated profile e 0,80 per la DM-SNR curve) sono estremamente correlate tra loro.

Essendo che i nostri parametri sono ricavati dall'applicazione di funzioni statistiche su due grandezze fisiche, era aspettato trovare due blocchi di forte correlazione all'interno della matrice.

Possiamo notare come la kurtosis e la skewness of the integrated profile siano estremamente importanti per determinare la target_class, mentre la mean e la standard deviation della DM-SNR curve sono meno importanti, ma comunque molto rilevanti. Anche la mean of integrated profile ha una correlazione significativa (-0,68), cioè ha una forte correlazione inversa.

Notiamo che nessun attributo ha un indice di correlazione con la target_class vicino allo 0, ciò significa che sono tutti importanti nella determinazione della classe. Possiamo notare che comunque possiamo eliminare i 3 attributi che hanno un indice di correlazione molto alto ($|M_{i,j}|>= 0,80$) tra loro.

####Dendrogramma

Un dendrogramma è un grafo ad albero utilizzato per raffigurare la disposizione dei cluster relativi alle features. L'obiettivo è quello di visualizzare somiglianze tra i cluster, rilevando potenziali caratteristiche duplicate e potenzialmente aiutando nella riduzione della dimensionalità.

Questo grafico indica la forza delle relazioni esistenti tra due cluster in base alla distanza che intercorre tra l'origine e la linea verticale più vicina che connette le linee orizzontali corrispondenti ai due elementi considerati.

In [ ]:
names = dataset.columns
inv_corr = 1 - corr # This is the 'dissimilarity' method

fig = ff.create_dendrogram(inv_corr, 
                           labels=names, 
                           colorscale=COLOR_PALETTE,
                           linkagefun=lambda x: hc.linkage(x, 'average'))

fig['layout'].update(dict(
    title="Dendrogramma di correlazione tra gli attributi",
    width=800, 
    height=600,
    xaxis=dict(
        title='Features',
    ),
    yaxis=dict(
        title='Distance',
    ),
))
py.iplot(fig, filename='dendrogram_corr_clustering')

DA RIVEDERE

Non avendo distanze piccole tra i vari cluster, possiamo rafforzare la nostra idea di mantenere tutti gli attributi durante la fase di classificazione.

###Suddivisione del dataset e normalizzazione dei dati

Per evitare "data leakage" è importante eseguire ogni trasformazione dopo la separazione dei dati. Il test set deve simulare dei dati reali acquisiti dopo l'addestramento del nostro modello, quindi non può essere utilizzato in nessun caso insieme al train set.

Molte volte i dataset contengono dei dati che variano molto rispetto alla media in termini di grandezza. Molti algoritmi di classificazione utilizzano la distanza euclidea per misurare la distanza tra due dati, quindi delle distanze troppo grandi possono causare dei problemi.

Per questo motivo, due features diverse possono influenzare in maniera diversa il classificatore in base all'unità di misura. Per esempio una features i cui valori sono dell'ordine delle migliaia (900, 5000, 2000) avrà un peso maggiore di una feature con valori nell'ordine delle unità (0,1, 10, 100) anche se in percentuale la variazione della prima risulterebbe essere minore rispetto alla seconda.

Per ovviare a questo problema utilizzeremo lo StandardScaler, che normalizzerà i nostri dati sottraendogli la media e dividendoli per lo scarto quadratico medio:
$x' = \frac{x - \mu_x}{\sigma_x}$

La normalizzazione verrà eseguita dopo la suddivisione dei dati, perchè il test set non deve essere influenzato in alcun modo dal train set e, eseguendo la normalizzazione sull'intero dataset, lo influenzerei tramite la media e la varianza del dataset intero.


In [ ]:
def normalize(X_set):
  scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
  return scaler.fit_transform(X_set)

X_dataset = dataset.drop(['target_class'], axis=1)
y_data = dataset['target_class']
X_train, X_test, y_train, y_test = train_test_split(X_dataset,y_data, test_size=0.2, random_state=42, stratify=y_data)
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)

<a name="PCA"></a>
##Principal Component Analysis

La Principal Component Analysis (PCA) è una tecnica che consente di ridurre la dimensionalità del dataset, proiettando negli assi principali gli attributi con maggior varianza.
La variazione di complessità avviene limitandosi ad analizzare le variabili con maggior varianza, quindi, più importanti.

Essendo che le componenti vengono inserite man mano in nuovi assi, si otterrà una base ortogonale nella quale tutte le componenti aggiunte saranno a loro volta ortogonali alle precedenti.



In [ ]:
p, x_train_PCA6 = PCA_Reduction(X_train_norm, plot = True , n_components = 6)
x_train_PCA4 = PCA_Reduction(X_train_norm, plot = False , n_components = 4)
p

In [ ]:
x_train_PCA6.head(5)

,PC#1,PC#2,PC#3,PC#4
0,-2.463495,-2.561296,0.358478,-0.816815
1,-0.221554,-0.398319,0.064316,0.146490
2,0.568718,0.347144,0.085768,-0.013019
3,-0.767799,-0.632460,-0.086334,0.430180
4,-2.056926,-1.624101,-0.212691,-0.535097


Quello che possiamo leggere da questo grafico è che già con le prime 4 componenti principali raggiungiamo una varianza cumulativa del 94,35%.

Possiamo ridurre la complessità del nostro dataset utilizzando solo le prime 4 componenti principali oppure mantenendo le prime 6.



<a name="bilanciamento"></a>
##Bilanciamento Dataset 

Esistono diverse tecniche per bilanciare il dataset:
- **oversampling:** tecnica che consente di creare dei campioni appartenenti alla classe con meno record;
- **undersampling:** tecnica che consente di rimuovere dei campioni appartenenti alla classe con più record;
- **class weight:** tecnica che consente di bilanciare il dataset inserendo un parametro chiamato "weight" all'interno del dataset, che contiene i pesi delle varie classi del dataset. Il peso viene assegnato in base alla proporzione tra il numero di parametri della classe più grande e il numero di parametri della classe più piccola. (per esempio W = 1 per la classe "0" e W = 10 per la classe "1") 
Durante la fase di train i vari record influenzeranno il modello in proporzione al peso assegnatogli.
Questa è la soluzione meno invasiva, perché non modifica il dataset ma solamente il modo in cui i record agiscono sul modello.
-**pipeline**: questa tecnica consiste nell'applicare sia l'oversampling sul dataset più piccolo, che l'undersampling sul dataset più grande.

Per bilanciare questo dataset ho optato per la tecnica della pipeline, nella quale procederò a modificare i record di entrambe le classi.

Il bilanciamento si deve eseguire solo sul train set, perché il test set deve essere più vicino possibile ai dati reali.



In [ ]:
x = x_train_PCA4
y = y_train

model = SVC(kernel='linear', probability=True, random_state=42)
scores = cross_val_score(model, x, y, scoring='roc_auc', cv=5, n_jobs=-1)
score = mean(scores)
print('Lo score calcolato sul dataset a cui è stata applicata la PCA mantenendo 4 componenti è:%.3f' % (score))

x = x_train_PCA6
model = SVC(kernel='linear', probability=True, random_state=42)
scores = cross_val_score(model, x, y, scoring='roc_auc', cv=5, n_jobs=-1)
score = mean(scores)
print('Lo score calcolato sul dataset a cui è stata applicata la PCA mantenendo 6 componenti è:%.3f' % (score))

x = X_train_norm
model = SVC(kernel='linear', probability=True, random_state=42)
scores = cross_val_score(model, x, y, scoring='roc_auc', cv=5, n_jobs=-1)
score = mean(scores)
print('Lo score calcolato sul dataset normalizzato è:%.3f' % (score))

x = x_train_PCA6

Lo score calcolato sul dataset a cui è stata applicata la PCA mantenendo 4 componenti è:0.935
Lo score calcolato sul dataset a cui è stata applicata la PCA mantenendo 6 componenti è:0.975
Lo score calcolato sul dataset normalizzato è:0.976


Applicando la PCA e tenendo le 6 features riusciamo a ridurre la dimensionalità del dataset senza perdere troppa risoluzione. Questo è dato dall'eliminazione di due features che nel complesso hanno una varianza dello 0,45%

In [ ]:
#Bilanciamento dataset. da fare solo sul training set
over_ratio = [0.3,0.4,0.5]
under_ratio = [0.7,0.6,0.5]
hi_score = 0
best_ratio = (0,0)
for o in over_ratio:
  for u in under_ratio:
    model = SVC(kernel='linear', probability=True, random_state=42)
    over = SMOTE(sampling_strategy=o)
    under = RandomUnderSampler(sampling_strategy=u)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_val_score(pipeline, x, y, scoring='roc_auc', cv=5, n_jobs=-1)
    score = mean(scores)
    if score > hi_score:
      hi_score = score
      best_ratio = (o,u)
      print("cambio ratio {0} {1} ".format(o,u))
    print('SMOTE oversampling rate:%.1f, Random undersampling rate:%.1f , Media ROC AUC: %.3f' % (o, u, score))
print("Il ratio che restituisce un punteggio maggiore è (oversampling: {0},undersampling: {1})".format(best_ratio[0],best_ratio[1]))

cambio ratio 0.3 0.7 
SMOTE oversampling rate:0.3, Random undersampling rate:0.7 , Media ROC AUC: 0.975
SMOTE oversampling rate:0.3, Random undersampling rate:0.6 , Media ROC AUC: 0.975
cambio ratio 0.3 0.5 
SMOTE oversampling rate:0.3, Random undersampling rate:0.5 , Media ROC AUC: 0.976
SMOTE oversampling rate:0.4, Random undersampling rate:0.7 , Media ROC AUC: 0.976
cambio ratio 0.4 0.6 
SMOTE oversampling rate:0.4, Random undersampling rate:0.6 , Media ROC AUC: 0.976
SMOTE oversampling rate:0.4, Random undersampling rate:0.5 , Media ROC AUC: 0.975
SMOTE oversampling rate:0.5, Random undersampling rate:0.7 , Media ROC AUC: 0.975
SMOTE oversampling rate:0.5, Random undersampling rate:0.6 , Media ROC AUC: 0.976
SMOTE oversampling rate:0.5, Random undersampling rate:0.5 , Media ROC AUC: 0.975
Il ratio che restituisce un punteggio maggiore è (oversampling: 0.4,undersampling: 0.6)


- **SMOTE** (Synthetic Minority Oversampling Technique), è una tecnica che seleziona due campioni appartenenti ad una classe che sono vicini tra loro, "traccia una linea" tra loro nello spazio delle features e crea un nuovo campione lungo questa linea.

- **Random Undersampling:** Random undersampling è una tecnica che consente semplicemente nel rimuovere dei campioni casuali della classe più popolosa.

- **Pipeline:** Tramite la pipeline è possibile effettuare in parallelo l'aumento di campione di una classe tramite SMOTE e la riduzione dell'altra classe tramite random undersampling.

- **ROC AUC Score:** Una ROC è una curva che mostra le performance del classificatore, tenendo conto del *True Positive Rate* e del *False Positive Rate*. AUC, "Area under the ROC Curve.", misura l'area sotto la ROC curve. Questo punteggio è quindi calcolato applicando la tecnica di AUC sulla ROC.   

- **SVC:** SVC (Support Vector Clustering) è un modello di apprendimento utilizzato per la classificazione, che estende il più famoso SVM (Support Vector Machine).

Il punteggo relativo ai differenti ratio è molto simile tra loro, compreso il punteggio calcolato con il ratio originale. La scelta di uno rispetto all'altro non è troppo rilevante, quindi ho deciso di prendere (0.4,0.6) perchè mediamente ha performato leggermente meglio degli altri.


<a name="classificazione"></a>
##Classificazione

In questa ultima parte proveremo vari modelli di classificazione sul nostro dataset per vedere quale si comporta meglio sui nostri dati.

In [ ]:
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle = True)

lr = LogisticRegression(random_state=42)
svm = SVC(probability=True, random_state=42)
nbc = GaussianNB()
rfc = RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier()

<a name="lr"></a>
###Logistic Regression

In [ ]:
score = grid_search_cv(lr, LR_PARAM, x_train_PCA6, y_train, kf)

<a name="svm"></a>
###Support Vector Machine

In [ ]:
score = grid_search_cv(svm, SVM_PARAM, x_train_PCA6, y_train, kf)

<a name="nbc"></a>
###Naïve Bayes Classifier

In [1]:
score = grid_search_cv(nbc, NBC_PARAM, x_train_PCA6, y_train, kf)

NameError: ignored

<a name="rfc"></a>
###Random Forest Classifier

In [ ]:
score = grid_search_cv(rfc, RFC_PARAM, x_train_PCA6, y_train, kf)

<a name="knn"></a>
###K-Nearest Neighbors Classifier

In [ ]:
score = grid_search_cv(knn, KNN_PARAM, x_train_PCA6, y_train, kf)

<a name="conclusioni"></a>
##Conclusioni